In [1]:
%load_ext autoreload
%autoreload 2

# 模块一演示：从“本地目录”读取 CYGNSS L1 v3.2 + GEE 可用性探测 + EnKF 配置 + LHS

本 Notebook 调用 `GNSSREnKFModule1`：
1. 读取 AOI（GeoJSON）；
2. 探测 GEE 数据可用性（IMERG / ERA5-Land / MODIS NDVI）；
3. **从本地目录递归读取 CYGNSS L1 v3.2**，抽取 DDM峰值反射率（线性）与入射角，统计每日覆盖与日均观测；
4. 在主程序中配置 EnKF 初始参数与 Q/R；
5. 生成 LHS 参数样本集合；
6. 导出制品到输出目录。

In [1]:
from GNSSREnKFModule2 import GNSSREnKFModule1
import os, pandas as pd

# ===== 用户需配置 =====
AOI_GEOJSON = "../boundary/zhengzhou.geojson"           # <-- 替换为你的AOI GeoJSON
LOCAL_CYGNSS_DIR = "/media/hl/Expansion/CYGNSS/CYGNSS_L1_V3.2/2021/7/"  # <-- 替换为“存放CYGNSS L1 v3.2”的目录
START_DATE = "2021-07-01"
END_DATE   = "2021-07-31"
OUTPUT_DIR = "./out_module1"

# ===== 运行模块一 =====
mod1 = GNSSREnKFModule1(
    aoi_geojson_path=AOI_GEOJSON,
    local_cygnss_dir=LOCAL_CYGNSS_DIR,
    start_date=START_DATE,
    end_date=END_DATE,
    output_dir=OUTPUT_DIR,
    min_hits_per_day=10,
    lhs_samples=50,
    random_seed=42,
    cygnss_required_substrings=("L1", "3.2"),     # 仅匹配文件名包含这两个子串，可按需调整
    cygnss_glob_pattern="**/*.nc"                 # 递归匹配 .nc；也可用 "**/*.nc4" 或 "**/*.h5"
)
results = mod1.run()
print("运行完成，制品目录：", OUTPUT_DIR)
os.listdir(OUTPUT_DIR)

运行完成，制品目录： ./out_module1


['enkf_config.json',
 'cygnss_daily_obs.csv',
 'availability_report.json',
 'daily_run_plan.csv',
 'lhs_params.csv']

## 每日运行计划（前10行）

In [ ]:
plan = results["daily_plan"]
plan.head(10)

## CYGNSS 日均观测（线性反射率/入射角，前10行）
（文件也已导出：`cygnss_daily_obs.csv`）

In [ ]:
cyg_daily = results["cygnss_daily_obs"]
cyg_daily.head(10)

## GEE 可用性与 EnKF 配置预览

In [ ]:
results["availability"], results["enkf_config"]

## LHS 采样参数（前5行）

In [ ]:
lhs = results["lhs_params"]
lhs.head(5)

## （可选）如果 ERA5/IMERG/NDVI 你将改为“本地数据”，可以更新回退标志并重新导出计划表。

In [ ]:
mod1.set_local_fallback(imerg=True, era5=True, ndvi=True)
mod1.export_artifacts()
mod1.daily_plan.head(10)